Find Metrics most correlated to SBS

In [14]:
#Import Modules
import sys
sys.path.append("..") #give this script access to all modules in parent directory
from pathlib import Path
import matplotlib.lines as mlines
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import math
from scipy.io import loadmat
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau
import tsfel
from operator import itemgetter
import os
from Data_Cleaning import preprocess
import Actigraph_Metrics
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import statistics
from sklearn.impute import SimpleImputer




In [3]:
# signal parameters
freq = 100 #signal is 100hz

Load All Patient Data

In [4]:
#Change to Load data of interest
#data_dir = 'C:/Users/sidha/OneDrive/Sid Stuff/PROJECTS/iMEDS Design Team/Data Analysis/PedAccel/data_analysis/PythonPipeline/PatientData'

data_dir = r'C:\Users\jakes\Documents\DT 6 Analysis\PythonCode\PedAccel\data_analysis\PythonPipeline\PatientData'


#set params
slice_size_min = 10
lead_time = 5
window_size = 100 #100 is 1 second worth of time

In [5]:


#Change for data of interest
#preprocess.load_and_segment_data(data_dir, slice_size_min, lead_time) #take sbs csv and accel gt3x to create a .mat file with vector magnitudes

Generate Most correlated features for some signal 

In [20]:
#There is no error handling in place, the .mat file must exist
for patient in os.listdir(data_dir):
        # filter out non-directories
        patient_dir = os.path.join(data_dir, patient)
        if os.path.isdir(patient_dir):
            data_filepath_vitals = os.path.join(patient_dir, f'{patient}_SICKBAY_{lead_time}MIN_{slice_size_min - lead_time}MIN.mat')
            data_filepath_accel = os.path.join(patient_dir, f'{patient}_{lead_time}MIN_{slice_size_min - lead_time}MIN.mat')           
       
        accel_data = loadmat(data_filepath_accel)
        x_mag = accel_data["x_mag"]
        accel_SBS = accel_data["sbs"]
        
        vitals_data = loadmat(data_filepath_vitals)
        hr = vitals_data['heart_rate']
        SpO2 = vitals_data['SpO2']
        rr = vitals_data['respiratory_rate']
        bps = vitals_data['blood_pressure_systolic']
        bpm = vitals_data['blood_pressure_mean']
        bpd = vitals_data['blood_pressure_diastolic']
        vitals_SBS = vitals_data['sbs']

        #Create a new cfg file for every metric to add to PCA
        MAD_cfg_file = tsfel.get_features_by_domain()

        hr_cfg_file = tsfel.get_features_by_domain()
        SpO2_cfg_file = tsfel.get_features_by_domain()
        rr_cfg_file = tsfel.get_features_by_domain()
        bps_cfg_file = tsfel.get_features_by_domain()
        bpm_cfg_file = tsfel.get_features_by_domain()
        bpd_cfg_file = tsfel.get_features_by_domain()

        # Extract features and restructure data
        MAD_features_list = []
        hr_features_list = []
        rr_features_list = []
        SpO2_features_list = []
        bpd_features_list = []
        bpm_features_list = []
        bps_features_list = []
        sbs_list = []

        for i in range(len(vitals_SBS)):
            #MAD = Actigraph_Metrics.VecMag_MAD(x_mag[i,:],100)

            #Replace with HR, this is a test and uses 2 sets of MAD data instead of MAD and HR
            
            hr_features = tsfel.time_series_features_extractor(hr_cfg_file, hr[i], fs=.5, verbose=0)
            SpO2_features = tsfel.time_series_features_extractor(SpO2_cfg_file, SpO2[i], fs=.5, verbose=0)
            rr_features = tsfel.time_series_features_extractor(rr_cfg_file, rr[i], fs=.5, verbose=0)
            bps_features = tsfel.time_series_features_extractor(bps_cfg_file, bps[i], fs=.5, verbose=0)
            bpm_features = tsfel.time_series_features_extractor(bpm_cfg_file, bpm[i], fs=.5, verbose=0)
            bpd_features = tsfel.time_series_features_extractor(bpd_cfg_file, bpd[i], fs=.5, verbose=0)

            hr_features_list.append(hr_features)
            SpO2_features_list.append(SpO2_features)
            rr_features_list.append(rr_features)
            bps_features_list.append(bps_features)
            bpm_features_list.append(bpm_features)
            bpd_features_list.append(bpd_features)

            sbs_list.append(vitals_SBS[i])


        #create these data frames for every new feature
        hr_columns = ['hr_feature_' + str(col) for col in hr_features_list[0]]
        hr_features_array = np.array(hr_features_list).reshape(-1, len(hr_columns))
        hr_df_features = pd.DataFrame(hr_features_array)
        hr_df_features.columns = hr_columns

        SpO2_columns = ['SpO2_feature_' + str(col) for col in SpO2_features_list[0]]
        SpO2_features_array = np.array(SpO2_features_list).reshape(-1, len(SpO2_columns))
        SpO2_df_features = pd.DataFrame(SpO2_features_array)
        SpO2_df_features.columns = SpO2_columns

        rr_columns = ['rr_feature_' + str(col) for col in rr_features_list[0]]
        rr_features_array = np.array(rr_features_list).reshape(-1, len(rr_columns))
        rr_df_features = pd.DataFrame(rr_features_array)
        rr_df_features.columns = rr_columns
    
        bps_columns = ['bps_feature_' + str(col) for col in bps_features_list[0]]
        bps_features_array = np.array(bps_features_list).reshape(-1, len(bps_columns))
        bps_df_features = pd.DataFrame(bps_features_array)
        bps_df_features.columns = bps_columns
    
        bpm_columns = ['bpm_feature_' + str(col) for col in bpm_features_list[0]]
        bpm_features_array = np.array(bpm_features_list).reshape(-1, len(bpm_columns))
        bpm_df_features = pd.DataFrame(bpm_features_array)
        bpm_df_features.columns = bpm_columns

        bpd_columns = ['bps_feature_' + str(col) for col in bpd_features_list[0]]
        bpd_features_array = np.array(bpd_features_list).reshape(-1, len(bpd_columns))
        bpd_df_features = pd.DataFrame(bpd_features_array)
        bpd_df_features.columns = bpd_columns

        
        df_sbs = pd.DataFrame({'SBS': sbs_list})
        
        #Concatenate all features and SBS scores
        df_features = pd.concat([hr_df_features, SpO2_df_features, rr_df_features, bps_df_features, bpm_df_features, bpd_df_features], axis =1)
        df = pd.concat([df_sbs, df_features], axis=1)

        #remove NaN values
        # Interpolate NaN values
        df_interpolated = df.interpolate()

        # Remove NaN values from DataFrame.values
        cleaned_values = df_interpolated.values[~np.isnan(df_interpolated.values).any(axis=1)]
        x = cleaned_values
        y = df['SBS'].values   
        
        # Normalize features
        features = np.hstack((hr_df_features.columns ,SpO2_df_features.columns,rr_df_features.columns,bps_df_features.columns,bpm_df_features.columns,bpd_df_features.columns))
        x = df_features.iloc[:, 1:].values
        x_normalized = StandardScaler().fit_transform(x)
        print(x)
        print(f'x mean should be 0 but is really: {np.mean((x_normalized))}')
        
        # Perform PCA Using All Features
        pca_actigraphy = PCA(n_components=4)
        principalComponents_actigraphy = pca_actigraphy.fit_transform(x_normalized)
        principal_actigraphy_Df = pd.DataFrame(data=principalComponents_actigraphy,
                                            columns=['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4'])

        print('Explained variation per principal component: {}'.format(pca_actigraphy.explained_variance_ratio_))

        # Plot PCA for each principal component
        for component in range(pca_actigraphy.n_components_ - 1):
            plt.figure(figsize=(8, 6))
            plt.xlabel(f'Principal Component - {component+1}', fontsize=12)
            plt.ylabel('Principal Component - {}'.format(component+2), fontsize=12)
            plt.title(f'Principal Component Analysis of Actigraphy and SBS\n for {patient}_{lead_time}MIN_{slice_size_min - lead_time}MIN with vitals data only', fontsize=14)
            
            for i in range(len(df_sbs)):
                if df['SBS'][i] == -1:
                    color = 'purple'
                elif df['SBS'][i] == 0:
                    color = 'blue'
                elif df['SBS'][i] == 1:
                    color = 'orange'
                elif df['SBS'][i] == 2:
                    color = 'red'
                plt.scatter(principal_actigraphy_Df.loc[i, f'principal component {component+1}'], 
                            principal_actigraphy_Df.loc[i, f'principal component {component+2}'], 
                            c=color, s=50)
            
            # Manually create a legend
            neg1 = mlines.Line2D([], [], color='purple', marker='o', ls='', label='SBS -1')
            zero = mlines.Line2D([], [], color='blue', marker='o', ls='', label='SBS 0')
            one = mlines.Line2D([], [], color='orange', marker='o', ls='', label='SBS 1')
            two = mlines.Line2D([], [], color='red', marker='o', ls='', label='SBS 2')
            plt.legend(handles=[neg1, zero, one, two])
            
            plt.show()

c:\Users\jakes\.virtualenvs\DT6Analysis\Lib\site-packages\tsfel\feature_extraction\features.py:498: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return scipy.stats.kurtosis(signal)
c:\Users\jakes\.virtualenvs\DT6Analysis\Lib\site-packages\tsfel\feature_extraction\features.py:518: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return scipy.stats.skew(signal)
c:\Users\jakes\.virtualenvs\DT6Analysis\Lib\site-packages\tsfel\feature_extraction\features.py:498: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return scipy.stats.kurtosis(signal)
c:\Users\jakes\.virtualenvs\DT6Analysis\Lib\site-packages\tsfel\feature_extra

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''